### Шаг 1. Загрузка и просмотр данных. 

In [1]:
import pandas as pd #импорт библиотеки pandas

df = pd.read_csv('/datasets/data.csv') # чтение файла csv и запись его в структуру Датафрейм
df.head(10) # вывод первых 10 строк датафрейма

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info() #общая информация о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df[df['dob_years']< 18]['dob_years'].value_counts() # поиск заемщиков младше 18 лет

0    101
Name: dob_years, dtype: int64

In [4]:
df[df['total_income'].isna() == True].info() # вывод общей информации о срезе данных с пропущенной информацией о доходах

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [5]:
df['gender'].value_counts() # поиск аномальных значений или ошибок в поле заемщика

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [6]:
df['children'].value_counts() # поиск аномальных значений или ошибок в информации о количестве детей заемщика
#df[df['children']==20]

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

### Вывод


В таблице из набора данных вместо трудового стажа и ежемесячного дохода встречается значение NaN. Возможно это связано с тем, что заемщик не предоставил банку документы с места работы или имел место технический сбой.

Так же в данных о стаже работы встречаются отрицательные и аномально большие значения, а в возрасте значения 0. Такие данные могли получиться из-за технических сбоев при выгрузке или особенностях расчета: 
- аномально большое значение является стажем работы в часах, а не в днях;
- отрицательное значение - расчет стажа относительно текущей даты.

Т.к. эти данные не предполагается анализировать в рамках запроса от Заказчика, оставим их в первоначальном виде.

В данных о поле клиента встречается значение 'XNA', но т.к. анализ зависимости между полом клиента и возвратом кредита в срок не предполагается, а такие данные, как количество детей, семейное положение, уровень дохода и цель кредита заполнены, то отсутствием значения о поле клиента можно пренебречь и не удалять из набора данных.

В информации о количестве детей встречается значение -1 и 20. Вероятно, это ошибка. Т.к. эти данные составляют всего 0.5%, то их можно удалить без искажения статистики.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
# нахождение среднего значения и медианы для каждого типа занятости 
median_employee = df[df['income_type']=='сотрудник']['total_income'].median() 
mean_employee = df[df['income_type']=='сотрудник']['total_income'].mean()
median_companion = df[df['income_type']=='компаньон']['total_income'].median()
mean_companion = df[df['income_type']=='компаньон']['total_income'].mean()
median_pensioner = df[df['income_type']=='пенсионер']['total_income'].median()
mean_pensioner = df[df['income_type']=='пенсионер']['total_income'].mean()
median_state = df[df['income_type']=='госслужащий']['total_income'].median()
mean_state = df[df['income_type']=='госслужащий']['total_income'].mean()
median_businessman = df[df['income_type']=='предприниматель']['total_income'].median()
mean_businessman = df[df['income_type']=='предприниматель']['total_income'].mean()

data = [['сотрудник', mean_employee, median_employee],
       ['компаньон', mean_companion, median_companion],
       ['пенсионер', mean_pensioner, median_pensioner],
       ['госслужащий', mean_state, median_state],
       ['предприниматель', mean_businessman, median_businessman]]
columns = ['income_type', 'mean', 'median']

income_research = pd.DataFrame(data=data, columns=columns) # вывод данных о среднем и медиане в таблицу
income_research

,income_type,mean,median
0,сотрудник,161380.260488,142594.396847
1,компаньон,202417.461462,172357.950966
2,пенсионер,137127.465690,118514.486412
3,госслужащий,170898.309923,150447.935283
4,предприниматель,499163.144947,499163.144947


In [8]:
# замена пропущенных значений о доходах медианой в зависимости от типа занятости
df.loc[(df['total_income'].isna()==True)&(df['income_type']=='сотрудник'), 'total_income'] = median_employee 
df.loc[(df['total_income'].isna()==True)&(df['income_type']=='компаньон'), 'total_income'] = median_companion
df.loc[(df['total_income'].isna()==True)&(df['income_type']=='пенсионер'), 'total_income'] = median_pensioner
df.loc[(df['total_income'].isna()==True)&(df['income_type']=='госслужащий'), 'total_income'] = median_state
df.loc[(df['total_income'].isna()==True)&(df['income_type']=='предприниматель'), 'total_income'] = median_businessman
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
# удаление срок, в которых содержатся ошибочные данные о наличии детей
df.loc[df['children']==-1, 'children'] = None
df.loc[df['children']==20, 'children'] = None
df.dropna(subset=['children'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null float64
days_employed       19240 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null float64
purpose             21402 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


In [10]:
df['children'] = df['children'].astype('int') # возвращение столбцу children типа данных int
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       19240 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null float64
purpose             21402 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Нам предстоит проанализировать зависимость между уровнем дохода и возвратом кредита в срок, поэтому необходимо заполнить значения NaN в поле 'total_income'. Т.к. уровень дохода напрямую зависит от типа занятости, а среднее и медианное отличаются значительно, то отсутствующие значения заменены медианным значением каждого из типов занятости.

Так же удалены строки с ошибочными данными о наличии детей.


### Замена типа данных

In [11]:
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median')).astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       21402 non-null int64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Для изменения типа данных был выбран метод astype, т.к. исходный тип данных float и данные не имели некорректных значений.

### Обработка дубликатов

In [12]:
df['education'] = df['education'].str.lower() # приведение данных об образовании к нижнему регистру
df.duplicated().sum() # подсчет количества дубликатов

71

In [13]:
df = df.drop_duplicates().reset_index(drop=True) # удаление дубликатов 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 12 columns):
children            21331 non-null int64
days_employed       21331 non-null int64
dob_years           21331 non-null int64
education           21331 non-null object
education_id        21331 non-null int64
family_status       21331 non-null object
family_status_id    21331 non-null int64
gender              21331 non-null object
income_type         21331 non-null object
debt                21331 non-null int64
total_income        21331 non-null int64
purpose             21331 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В наборе данных обнаружен 71 дубликат. Для их удаления выбран метод drop_duplicates() совместно с методом reset_index(), чтобы обновить индексацию после удаления дубликатов. Аргумент drop=True указывает, что включать столбец со старыми индексами в новый датафрейм не нужно.
Появление дубликатов может быть связано с техническим сбоем во время записи или выгрузки данных, или возникнуть при консолидации данных из разных источников.

### Лемматизация

In [14]:
df['purpose'].unique() # вывод списка уникальных значений целей кредита

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [15]:
from pymystem3 import Mystem # импорт библиотеки Mystem для лемматизации
m = Mystem()

def change_purpose(purpose): # функция для приведения целей кредита к единообразности
    if 'коммерческий' in m.lemmatize(purpose['purpose']):
        return 'покупка или реконструкция коммерческой недвижимости'
    if 'ремонт'  in m.lemmatize(purpose['purpose']) and 'жилье' in m.lemmatize(purpose['purpose']):
        return 'ремонт жилья'
    if 'недвижимость' in m.lemmatize(purpose['purpose']):
        return 'покупка или строительство жилой недвижимости'
    if 'жилье' in m.lemmatize(purpose['purpose']):
        return 'покупка или строительство жилой недвижимости'
    if 'автомобиль' in m.lemmatize(purpose['purpose']):
        return 'покупка автомобиля'
    if 'образование' in m.lemmatize(purpose['purpose']):
        return 'получение образования'
    if 'свадьба' in m.lemmatize(purpose['purpose']):
        return 'свадьба' 
    
df['purpose'] = df.apply(change_purpose, axis=1)

In [16]:
df['purpose'].unique()

array(['покупка или строительство жилой недвижимости',
       'покупка автомобиля', 'получение образования', 'свадьба',
       'покупка или реконструкция коммерческой недвижимости',
       'ремонт жилья'], dtype=object)

### Вывод

Цели получения кредита сведены к следующему списку:
- покупка или строительство жилой недвижимости
- покупка или реконструкция коммерческой недвижимости
- ремонт жилья
- покупка автомобиля
- получение образования
- свадьба

Это сделано с помощью функции change_purpose, которая получает на вход датафрейм, лемматизирует столбец purpose и в зависимости от выполненного условия возвращает новое значение. Таким образом, 
- если "коммерческий" входит в список лемм, то новая цель получения кредита "покупка или реконструкция коммерческой недвижимости"
- если "ремонт" и "жилье" входит в список лемм, то новая цель получения кредита "ремонт жилья"
- если "недвижимость" или "жилье" входит в список лемм, то новая цель получения кредита "покупка или строительство жилой недвижимости"
- если "автомобиль" входит в список лемм, то новая цель получения кредита "покупка автомобиля"
- если "образование" входит в список лемм, то новая цель получения кредита "получение образования"
- если "свадьба" входит в список лемм, то новая цель получения кредита "свадьба"

Порядок важен!

### Категоризация данных

In [17]:
# выделение словаря с семейным положением 
dict_family_status = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
dict_family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [18]:
df['total_income'].quantile([0.25, 0.5, 0.75]) # нахождение 0.25, 0.5 и 0.75 квантилей уровня дохода

0.25    107507.0
0.50    142594.0
0.75    195842.0
Name: total_income, dtype: float64

In [19]:
#  функция, разделяющая клиентов на группы по уровню дохода в соответствии с найденными квантилями
def income_group(data):
    if data < 107625:
        return 1
    if 107623 <= data < 142594:
        return 2
    if 142594 <= data < 195820:
        return 3
    if data >= 195820:
        return 4
        
df['income_group'] = df['total_income'].apply(income_group)

In [20]:
# формирование датайфрейма для анализа
df = df[['children', 'family_status_id', 'purpose', 'income_group', 'debt']]
df

,children,family_status_id,purpose,income_group,debt
0,1,0,покупка или строительство жилой недвижимости,4,0
1,1,0,покупка автомобиля,2,0
2,0,0,покупка или строительство жилой недвижимости,3,0
3,3,0,получение образования,4,0
4,0,1,свадьба,3,0
...,...,...,...,...,...
21326,1,1,покупка или строительство жилой недвижимости,4,0
21327,0,0,покупка автомобиля,3,0
21328,1,1,покупка или строительство жилой недвижимости,1,1
21329,3,0,покупка автомобиля,4,1


### Вывод


В отдельный словарь выделен список с семейным положением клиентов, т.к. осуществлять группировку по id быстрее, чем по текстовому полю.

Так же клиенты разделены на 4 группы по уровню дохода в соответствии с 0.25, 0.5 и 0.75 квантилями.

Датафрейм состоит из столбцов, необходимых для ответов на вопросы Закачика (children, family_status_id, purpose, income_group, debt)


### Шаг 3. Исследование критериев, влияющих на возврат кредита

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [21]:
# подсчет общего кол-ва заемщиков и заемщиков, которые имели задолженность в группировке по количеству детей
children_group = df.groupby('children').agg({'debt': ['count', 'sum']}) 
children_group = pd.DataFrame(children_group['debt']) # изменеие структуры данных на датафрейм
children_group['ratio'] = children_group['sum'] / children_group['count'] * 100 # расчет доли заемщиков, которые имели задолженность по кредитам
children_group

,count,sum,ratio
children,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


### Вывод

Наиболее надежны для банка клиенты, у которых нет детей. Заемщики с тремя детьми находятся в средней группе риска. Возможно, это связано со льготами, которые получают многодетные семьи. В остальных случаях риск возникновения задолженности перед банком выше 9,2%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# подсчет общего кол-ва заемщиков и заемщиков, которые имели задолженность в группировке по семейному положению
family_status_group = df.groupby('family_status_id').agg({'debt': ['count', 'sum']}) 
family_status_group = pd.DataFrame(family_status_group['debt']) # изменеие структуры данных на датафрейм
family_status_group['ratio'] = family_status_group['sum'] / family_status_group['count'] * 100 # расчет доли заемщиков, которые имели задолженность по кредитам
family_status_group = family_status_group.merge(dict_family_status, on='family_status_id', how='left')
family_status_group[['family_status', 'count', 'sum', 'ratio']]

,family_status,count,sum,ratio
0,женат / замужем,12261,927,7.560558
1,гражданский брак,4134,385,9.313014
2,вдовец / вдова,951,63,6.624606
3,в разводе,1189,84,7.064760
4,Не женат / не замужем,2796,273,9.763948


### Вывод

Чаще всего имеют задолженности клиенты, не состоявшие в официальном браке. Вероятно, это люди молодого возраста, которые могут иметь невысокий или нестабильный доход.

Для банка заемщики в официальном браке или в разводе чуть более надежны среднего показателя в 8%. Скорее всего сюда входят лица средней возрастной категории, имеющие стабильный доход и финансовую поддержку второго супруга.

Самые надежные клиенты в статусе "вдовец/вдова". Вероятно это люди старшей возрастной категории, которые имеют стабильный доход и финансовую поддержку детей.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
# подсчет общего кол-ва заемщиков и заемщиков, которые имели задолженность в группировке по уровню дохода
income_group = df.groupby('income_group').agg({'debt': ['count', 'sum']}) 
income_group = pd.DataFrame(income_group['debt']) # изменеие структуры данных на датафрейм
income_group['ratio'] = income_group['sum'] / income_group['count'] * 100 # расчет доли заемщиков, которые имели задолженность по кредитам
income_group

,count,sum,ratio
income_group,,,
1,5340,427,7.996255
2,4377,382,8.727439
3,6279,542,8.631948
4,5335,381,7.141518


### Вывод

Доход свыше 195 тыс руб. уменьшает риск возникновения задолженности до 7%.

В средней гуппе риска находятся заемщики, доход которых ниже 107 тыс руб. Вероятно это объясняется небольшой суммой кредита.

Клиенты среднего уровня дохода чаще всего имеют задолженности перед банком, т.к. чаще берут большие кредиты, например, на покупку недвижимости или предметы роскоши (дорогие автомобили).

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
# подсчет общего кол-ва заемщиков и заемщиков, которые имели задолженность в зависимости от цели кредита
purose_group = df.groupby('purpose').agg({'debt': ['count', 'sum']}) 
purose_group = pd.DataFrame(purose_group['debt']) # изменеие структуры данных на датафрейм
purose_group['ratio'] = purose_group['sum'] / purose_group['count'] * 100 # расчет доли заемщиков, которые имели задолженность по кредитам
purose_group

,count,sum,ratio
purpose,,,
покупка автомобиля,4279,400,9.347978
покупка или реконструкция коммерческой недвижимости,1303,98,7.521105
покупка или строительство жилой недвижимости,8844,647,7.315694
получение образования,3988,369,9.252758
ремонт жилья,604,35,5.794702
свадьба,2313,183,7.911803


### Вывод

Наиболее рискованно для банка выдавать кредит на покупку автомобиля и получение образования. Ведь автомобиль может сломаться или попасть в аварию, что снизит платежеспособность клиента, или быть угнан. Так же цель кредита "получение образования" предполагает, что заемщик не сможет работать полноценно, т.к. не имеет профессии и времени на работу.

Кредит на ремонт жилья самый надежный. Скорее всего ввиду небольшой суммы займа.

В средней группе риска кредиты на приобретение недвижимости и проведение свадьбы. Покупка коммерческой недвижимости предполагает получение дохода от ее использования, что увеличивает платежеспособность клиента. Свадьба и покупка собственного жилья - начало семейной жизни. Как показал анализ зависимости семейного положения и задолженностей, семейные люди достаточно платежеспособны.

### Шаг 4. Общий вывод

Был проведен первичный анализ исходных данных, их предобработка, а затем исследование.

В информации о стаже работы и уровне дохода найдены отсутствующие значения. Для их заменены выбраны медианы в зависимости от типа занятости клиента, т.к. среднее арифметическое имеет значительное смещение относительно середины, а доход напрямую зависит от типа занятости.

Так же в данных о стаже работы встречаются отрицательные и аномально большие значения и приведены к одной единице измерения. Такие данные могли получиться из-за особенностей расчета в разных источниках данных: 
- аномально большое значение является стажем работы в часах, а не в днях;
- отрицательное значение - расчет стажа относительно текущей даты.

В поле о наличии детей обнаружены некорректные и аномальные значения (-1 и 20). Их можно было заменить на среднее или медианное значение, но т.к. ошибочные данные составляют менее 0,5% от всего набора данных, то были удалены без искажения статистики.

В наборе данных обнаружен 71 дубликат. Возможно они возникли при консолидации данных из разных источников или из-за технического сбоя при выгрузке.

Для категоризации клиентов по уровню дохода данные были разделены на 4 группы в соответствии с 0.25, 0.5, 0.75 квантиялми. Таким образом:
- 1 группа - доход ниже 107 тыс руб
- 2 группа - доход от 107 тыс руб до 142 тыс руб
- 3 группа - доход от 142 тыс руб до 195 тыс руб
- 4 группа - доход выше 195 тыс руб

В результате исследования получены следующие выводы:

Наиболее надежны для банка клиенты, у которых нет детей. Заемщики с тремя детьми находятся в средней группе риска. Возможно, это связано со льготами, которые получают многодетные семьи. В остальных случаях риск возникновения задолженности перед банком выше 9,2%.

Чаще всего имеют задолженности клиенты, не состоявшие в официальном браке. Вероятно, это люди молодого возраста, которые могут иметь невысокий или нестабильный доход.

Для банка заемщики в официальном браке или в разводе чуть более надежны среднего показателя в 8%. Скорее всего сюда входят лица средней возрастной категории, имеющие стабильный доход и финансовую поддержку второго супруга.

Самые надежные клиенты в статусе "вдовец/вдова". Вероятно это люди старшей возрастной категории, которые имеют стабильный доход и финансовую поддержку детей.

Доход свыше 195 тыс руб. уменьшает риск возникновения задолженности до 7%.

В средней гуппе риска находятся заемщики, доход которых ниже 107 тыс руб. Вероятно это объясняется небольшой суммой кредита.

Клиенты среднего уровня дохода чаще всего имеют задолженности перед банком, т.к. чаще берут большие кредиты, например, на покупку недвижимости или предметы роскоши (дорогие автомобили).

Наиболее рискованно для банка выдавать кредит на покупку автомобиля и получение образования. Ведь автомобиль может сломаться или попасть в аварию, что снизит платежеспособность клиента. Так же цель кредита "получение образования" предполагает, что заемщик не сможет работать полноценно, т.к. не имеет профессии и времени на работу. 

Кредит на ремонт жилья самый надежный. Скорее всего ввиду небольшой суммы займа.

В средней группе риска кредиты на приобретение недвижимости и проведение свадьбы. Покупка коммерческой недвижимости предполагает получение дохода от ее использования, что увеличивает платежеспособность клиента. Свадьба и покупка собственного жилья - начало семейной жизни. Как показал анализ зависимости семейного положения и задолженностей, семейные люди достаточно платежеспособны.

Примерно в 8% случаев клиенты не возвращают кредит в срок. Заемщики, берущие кредит на покупку автомобиля или получение образования, не состовшие в официальном браке, в группе повышенного риска по возникновению задолженностей. Самые надежные клиенты состоят в браке, имеют доход выше 195000 руб, берут кредит на покупку или ремонт жилья или приобретают коммерческую недвижимость.